In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Brand Voice using Tuned foundation model

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


Your brand's voice is its soul - the way it speaks to the world. This notebook will become your essential toolkit for crafting and refining a distinct voice of the brand for all your content creation efforts. It's designed to be a living document, guiding you in translating abstract brand values into tangible communication.

On Vertex AI, tuning allows you to customize a foundation model for more specific tasks or knowledge domains.

While the prompt design is excellent for quick experimentation, if training data (examples) is available, tuning a model enables you to customize the model for the characterstics of your brand you want to project.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

### Objective

This tutorial teaches you how to tune a foundational model on new unseen data and you will use the following Google Cloud products:

- Vertex AI Generative AI Studio
- Vertex AI Pipelines
- Vertex AI Model Registry
- Vertex AI Endpoints

The steps performed include:

- Upload training data
- Create a pipeline job
- Inspect your model on Vertex AI Model Registry
- Get predictions from your tuned model

### Quota
**important**: Tuning the text-bison@002  model uses the tpu-v3-8 training resources and the accompanying quotas from your Google Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need to request additional quota via the [Quotas page](https://console.cloud.google.com/iam-admin/quotas).

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install Vertex AI SDK

In [4]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


**Colab only:** Uncomment the following cell to restart the kernel or use the restart button. For Vertex AI Workbench you can restart the terminal using the button on top.

In [5]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [1]:
from google.colab import auth

auth.authenticate_user()

### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`. Otherwise, check the support page: Locate the [project ID](https://support.google.com/googleapi/answer/7014113). Please update `PROJECT_ID` below.

In [2]:
PROJECT_ID = "YOUR_GOOGLE_CLOUD_PROJECT_HERE"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Create a bucket
Now you have to create a bucket that we will use to store our tuning data. To avoid name collisions between users on resources created, you generate a UUID for each instance session and append it to the name of the resources you create in this tutorial.

In [11]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

Choose a bucket name and update the `BUCKET_NAME` parameter.

In [14]:
BUCKET_NAME = "genai-mkt-dev/tune-dataset"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"  # @param {type: "string"}

In [9]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "<your-bucket-name>":
    BUCKET_NAME = "vertex-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if your bucket doesn't already exist: Run the following cell to create your Cloud Storage bucket.

In [15]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

CommandException: The mb command requires a URL that specifies a bucket.
"gs://genai-mkt-dev/tune-dataset" is not valid.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [16]:
! gsutil ls -al $BUCKET_URI

         0  2024-02-27T08:10:46Z  gs://genai-mkt-dev/tune-dataset/#1709021446382005  metageneration=1
     50342  2024-02-27T08:14:13Z  gs://genai-mkt-dev/tune-dataset/tune_data_brand_voice.jsonl#1709021653614330  metageneration=1
     20866  2024-02-27T08:28:23Z  gs://genai-mkt-dev/tune-dataset/tune_eval_data_brand_voice.jsonl#1709022503861849  metageneration=1
                                 gs://genai-mkt-dev/tune-dataset/genai-tuned-model-khzprkhd/
TOTAL: 3 objects, 71208 bytes (69.54 KiB)


### Import libraries

**Colab only**: Run the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.

In [17]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

In [18]:
from typing import Union

import pandas as pd

from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel

## Tune your Model

Now it's time for you to create a tuning job. Tune a foundation model by creating a pipeline job using Generative AI Studio, cURL, or the Python SDK. In this notebook, we will be using the Python SDK. You will be using a Q&A with a context dataset in JSON format.

### Training Data
💾 Your model tuning dataset must be in a JSONL format where each line contains a single training example. You must make sure that you include instructions.

Upload to cloud storage bucket and add filenames below


In [19]:
training_data_filename = "tune_data_brand_voice.jsonl"

In [20]:
evaluation_data_filename = "tune_eval_data_brand_voice.jsonl"

You can check to make sure that the files are available in your Google Cloud Storage bucket:

In [21]:
! gsutil ls -al $BUCKET_URI

         0  2024-02-27T08:10:46Z  gs://genai-mkt-dev/tune-dataset/#1709021446382005  metageneration=1
     50342  2024-02-27T08:14:13Z  gs://genai-mkt-dev/tune-dataset/tune_data_brand_voice.jsonl#1709021653614330  metageneration=1
     20866  2024-02-27T08:28:23Z  gs://genai-mkt-dev/tune-dataset/tune_eval_data_brand_voice.jsonl#1709022503861849  metageneration=1
                                 gs://genai-mkt-dev/tune-dataset/genai-tuned-model-khzprkhd/
TOTAL: 3 objects, 71208 bytes (69.54 KiB)


In [22]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"
EVAUATION_DATA_URI = f"{BUCKET_URI}/{evaluation_data_filename}"

### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.

#### Recommended Tuning Configurations
✅ Here are some recommended configurations for tuning a foundation model based on the task, in this example Q&A. You can find more in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

Extractive QA:
- Make sure that your train dataset size is 100+
- Training steps [100-500]. You can try more than one value to get the best performance on a particular dataset (e.g. 100, 200, 500)

In [23]:
# create tensorboard
display_name = "Adapter tuning Voice - "

tensorboard = aiplatform.Tensorboard.create(
    display_name=display_name,
    project=PROJECT_ID,
    location=REGION,
)

print(tensorboard.display_name)
print(tensorboard.resource_name)

INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Creating Tensorboard
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Create Tensorboard backing LRO: projects/882920967572/locations/us-central1/tensorboards/432877727855411200/operations/6991465721410617344
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Tensorboard created. Resource name: projects/882920967572/locations/us-central1/tensorboards/432877727855411200
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:To use this Tensorboard in another session:
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:tb = aiplatform.Tensorboard('projects/882920967572/locations/us-central1/tensorboards/432877727855411200')


Adapter tuning Voice - 
projects/882920967572/locations/us-central1/tensorboards/432877727855411200


In [24]:
# Get tensorboard_id thats used in the pipeline
tensorboard_id = tensorboard.resource_name.split("tensorboards/")[-1]
print(tensorboard_id)

432877727855411200


In [25]:
MODEL_NAME = f"genai-tuned-model-{UUID}"
TRAINING_STEPS = 100

In [26]:
pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "large_model_reference": "text-bison@002",
    "project": PROJECT_ID,
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": 20,
    "evaluation_data_uri": EVAUATION_DATA_URI,
    "tensorboard_resource_id": tensorboard_id,
}

pipeline_root = f"{BUCKET_URI}/{MODEL_NAME}"
template_path = "https://us-kfp.pkg.dev/ml-pipeline/large-language-model-pipelines/tune-large-model/v2.0.0"

In [27]:
# Function that starts the tuning job
def tuned_model(
    project_id: str,
    location: str,
    template_path: str,
    model_display_name: str,
    pipeline_arguments: str,
):
    """Prompt-tune a new model, based on a prompt-response data.

    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with two keys, for
    example:
      {
        "input_text": <input prompt>,
        "output_text": <associated output>
      },

    Args:
      project_id: GCP Project ID, used to initialize aiplatform
      location: GCP Region, used to initialize aiplatform
      template_path: path to the template
      model_display_name: Name for your model.
      pipeline_arguments: arguments used during pipeline runtime
    """

    aiplatform.init(project=project_id, location=location)

    from google.cloud.aiplatform import PipelineJob

    job = PipelineJob(
        template_path=template_path,
        display_name=model_display_name,
        parameter_values=pipeline_arguments,
        location=REGION,
        pipeline_root=pipeline_root,
        enable_caching=True,
    )

    return job

Next, it's time to start your tuning job.

**Disclaimer:** tuning and deploying a model takes time.

In [28]:
job = tuned_model(PROJECT_ID, REGION, template_path, MODEL_NAME, pipeline_arguments)

In [29]:
job.submit()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/882920967572/locations/us-central1/pipelineJobs/tune-large-model-20240328232057
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/882920967572/locations/us-central1/pipelineJobs/tune-large-model-20240328232057')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tune-large-model-20240328232057?project=882920967572


Following the link above, you can view your pipeline run. As you can see in the screenshot below, it will execute the following steps:

- Validation
- Export managed dataset
- Convert JSONL to TFRecord
- Large language model tuning
- Upload LLM Model

`job.state` lets you check the state of your pipeline.

In [35]:
job.state

<PipelineState.PIPELINE_STATE_PENDING: 2>

## View your tuned foundational model on Vertex AI Model registry
When your tuning job is finished, your model will be available on Vertex AI Model Registry. The following Python SDK sample shows you how to list tuned models.

In [32]:
def list_tuned_models(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@002")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

In [ ]:
list_tuned_models(PROJECT_ID, REGION)

['projects/244831775715/locations/us-central1/models/2124430187700420608']


You can also use the Google Cloud Console UI to view all of your model in [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models). Below you can see an example of a tuned foundational model available on Vertex AI Model Registry.

## Use your tuned model to get predictions
Now it's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model registry.

In [ ]:
def fetch_model(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@002")
    list_tuned_models = model.list_tuned_model_names()
    tuned_model = list_tuned_models[0]

    return tuned_model

In [ ]:
deployed_model = fetch_model(PROJECT_ID, REGION)
deployed_model = TextGenerationModel.get_tuned_model(deployed_model)

Now you can start send a prompt to the API. Feel free to update the following prompt.

In [ ]:
PROMPT = """
Theme: Sales of new women's handbags at Cymbal\nUsing the Brand Voice, generate a personalized email with the theme mentioned above for the user
"""

In [ ]:
print(deployed_model.predict(PROMPT))

MultiCandidateTextGenerationResponse(text=" Subject: Discover the Latest Trends in Women's Handbags at Cymbal\n\nDear [User Name],\n\nWe're thrilled to announce the arrival of our new collection of women's handbags at Cymbal.\n\nOur designers have created a stunning array of styles, from classic totes to chic crossbody bags, each crafted with the finest materials and attention to detail.\n\nWhether you're looking for a timeless piece to complement your everyday wardrobe or a statement-making accessory for a special occasion, you'll find the perfect handbag at Cymbal.\n\nVisit our website or your nearest Cymbal store today to browse the collection and find your", _prediction_response=Prediction(predictions=[{'citationMetadata': {'citations': []}, 'safetyAttributes': {'blocked': False, 'categories': ['Derogatory', 'Insult', 'Profanity', 'Sexual'], 'scores': [0.1, 0.1, 0.1, 0.3], 'safetyRatings': [{'probabilityScore': 0.0, 'category': 'Dangerous Content', 'severityScore': 0.0, 'severity':

Compare with Foundation Model Prediction

In [ ]:
foundation_model = TextGenerationModel.from_pretrained("text-bison@002")
print(foundation_model.predict(PROMPT))

MultiCandidateTextGenerationResponse(text=' Subject: Discover the Latest Trends in Women\'s Handbags at Cymbal\n\nDear [User\'s Name],\n\nWe hope this email finds you well. As a valued customer of Cymbal, we\'re excited to share with you our latest collection of women\'s handbags.\n\nOur team has carefully curated a selection of stylish and functional handbags that cater to the modern woman\'s needs. From classic leather totes to trendy crossbody bags, we have something for every occasion.\n\nHere are a few highlights from our new collection:\n\n- The "Amelia" tote bag: This spacious and versatile tote is perfect for everyday use. It', _prediction_response=Prediction(predictions=[{'citationMetadata': {'citations': []}, 'safetyAttributes': {'safetyRatings': [{'severity': 'NEGLIGIBLE', 'probabilityScore': 0.0, 'severityScore': 0.0, 'category': 'Dangerous Content'}, {'severity': 'NEGLIGIBLE', 'severityScore': 0.0, 'category': 'Harassment', 'probabilityScore': 0.1}, {'severity': 'NEGLIGIBL